<a href="https://colab.research.google.com/github/amirpaia/blenderbot/blob/main/blender_finetuning_with_reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.Installing prerequisites

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Jun  7 07:54:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
from google.colab import drive
drive.mount('/content/drive')
mydrive_path = '/content/drive/MyDrive/colabs/'

Mounted at /content/drive


In [4]:
# !pip uninstall -q parlai
!pip install -q parlai
!pip install transformers

     |████████████████████████████████| 1.7 MB 7.9 MB/s 
     |████████████████████████████████| 248 kB 70.4 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 125 kB 52.2 MB/s 
     |████████████████████████████████| 76 kB 5.1 MB/s 
     |████████████████████████████████| 2.8 MB 54.9 MB/s 
     |████████████████████████████████| 2.7 MB 53.4 MB/s 
     |████████████████████████████████| 6.6 MB 36.9 MB/s 
     |████████████████████████████████| 46 kB 3.6 MB/s 
     |████████████████████████████████| 346 kB 36.4 MB/s 
     |████████████████████████████████| 208 kB 58.8 MB/s 
     |████████████████████████████████| 181 kB 62.5 MB/s 
     |████████████████████████████████| 235 kB 68.3 MB/s 
     |████████████████████████████████| 56 kB 5.0 MB/s 
     |████████████████████████████████| 40 kB 6.0 MB/s 
     |██████████████████████

# 1.Preparing Data

## Genreal Functions

In [5]:
def transfer_list_of_turns_to_dialog(d):
    if len(d)%2 !=0: d = d[:-1]
    t = ""
    for i in range(0,len(d),2):
        u1 = d[i]
        u2 = d[i+1]

        if (i+2) != len(d):
            t += "text:"+u1+"\t"+"labels:"+u2+"\n"
        else:
            t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"
    return t

def transfer_list_of_pairs_to_dialog(d):
  t = ""
  for i, text_label_pair in enumerate(d):
    u1 = text_label_pair[0]
    u2 = text_label_pair[1]

    if i != (len(d) - 1):
      t += "text:"+u1+"\t"+"labels:"+u2+"\n"
    else:
      t += "text:"+u1+"\t"+"labels:"+u2+"\t"+"episode_done:True"+"\n"

  return t

def convert_parlai_format_to_list_of_turns(lines):
    result = []
    for line in lines:
        text_label = line.split("\t")
        result.append(text_label[0].replace("text:", ""))
        result.append(text_label[1].replace("labels:", "").replace("\n",""))
    return result

t = ['hello','how are you','good','bye','test']
print(transfer_list_of_turns_to_dialog(t))

t = [['hello','how are you'],['good','bye']]
print(transfer_list_of_pairs_to_dialog(t))

text:hello	labels:how are you
text:good	labels:bye	episode_done:True

text:hello	labels:how are you
text:good	labels:bye	episode_done:True



## French Reddit

In [6]:
data_path = f"{mydrive_path}aliae-workspace/datasets/french_reddit/"
data_path

'/content/drive/MyDrive/colabs/aliae-workspace/datasets/french_reddit/'

In [7]:
with open(f"{data_path}/data_train.txt") as f:
    lines = f.readlines()
lines[:10]

["text:Pour ceux qui jouent à LoL et qui attendaient impatiemment le URF, ça sera au final le NURF qui sera là. L'inverse du Urf : cooldown augmentés, conso mana/énergie augmentée, dégâts des minions augmentés... Voilà, enjoy, moi je vais pleurer dans un coin parce que le urf était le mode qui me poussait à garder LoL installé. ^^^^quelqu'un ^^^^n'aime ^^^^pas ^^^^LoL ^^^^? ^^^^:)\tlabels:Ça, ou alors demain c'est le 1^er avril.\n",
 "text:Le vrai mode URF était déjà une blague du 1er avril. Sauf que ça a bien marché et ils ont laissé le mode 2 semaines.On verra bien demain de toute façon :(\tlabels:Et tu penses vraiment qu'ils vont remplacer un mode de jeu que tout le monde attend depuis 1 an par un truc pas fun du tout ?C'est clairement une blague.\n",
 "text:Riot est tellement une vaste blague comme entreprise qu'ils en seraient capable D:\tlabels:Une vaste blague comme entreprise ? Les mecs ont réussi, en pompant le concept d'un jeu existant à créer le jeu en ligne le plus joué au 

# 2.Finetuning

In [8]:
finetuned_model_path = f'{mydrive_path}blender-models/finetuned-reddit-90m/'
# init_model = 'zoo:blender/blender_400Mdistill/model'
# dict_file  = 'zoo:blender/blender_400Mdistill/model.dict'
init_model = 'zoo:blender/blender_90M/model'
dict_file  = 'zoo:blender/blender_90M/model.dict'
finetuned_model_path

'/content/drive/MyDrive/colabs/blender-models/finetuned-reddit-90m/'

In [ ]:
# 90M settings
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    # task= "french_blended_skill_talk,french_xpersona,french_empathetic_dialogues",
    # multitask_weights= "1,3,3",

    task='fromfile:parlaiformat', 
    fromfile_datapath= f'{data_path}data',
    fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}model',
    
    # initialize with a pretrained model
    init_model= init_model,
    dict_file=dict_file,
    
    # arguments we get from the pretrained model.
    # Unfortunately, these must be looked up separately for each model.
    n_heads=16, n_layers=8, n_positions=512, text_truncate=512,
    label_truncate=128, ffn_size=2048, embedding_size=512,
    activation='gelu', variant='xlm',
    dict_lower=True, dict_tokenizer='bpe',
    
    # depend on your gpu. 
    validation_every_n_epochs=0.25,
    num_epochs = 5,
    log_every_n_secs= 60,
    verbose = True,
    batchsize= 8, 
    fp16= True, fp16_impl= "mem_efficient",
    
    # arguments we get from the pretrained model.
    
    # speeds up validation
    skip_generation=True,
    vp= 10,
    validation_metric= "ppl", #vmt = "ppl"
    validation_metric_mode= "min", # vmm= "min"
    
    # helps us cram more examples into our gpu at a time
    dynamic_batching='full',

    
    # some training arguments, specific to this fine-tuning
    lr=1e-5, optimizer='adam',
    attention_dropout= 0.0, 
    model_parallel= False,
    warmup_updates=100,

    # customized parameters
    # inference= "beam"
    # beam_min_length= 20,
    # beam_block_ngram= 3,
    # beam_context_block_ngram= 3,
    # beam_size= 10,
)


                                 /@&%###%&&@@#
                      .,*/((((##@@@&%%#%%&&@@@&%%#/*.
             #@@&&&%%%%##(((///*****//(((###%%%&&&@@@@@&&%#%%#.
         .%&@@@@@&&&%%%####((((////((((####%%%&&&@@@@@&&%%#%%####,
           ./,,#(//**,,.....,,,,***////((((########%%%%%%%%###(((
              /*(//**,,,....,,,,***////((((########%%%%%%%%###(#%*
               (*,...      ...,,,***//////((((((///////(/*...,/#@@@(
               **,,..         ...,,,,,,,,,,........,,*///*...*(#@@@@&&*
               ./,,..          ...,,,,,,,,,........,,*//*,...*#/,,,,,/%#
                (*,..          ...,,,,,,,,,........,,*//*,..,/(      .,#(
                **,..          ...,,,,,,,,,.........,*//*,..,((       .,(#
                 /*,..          ....,,,,,,,.....  ..,***,,,,(#         ..#&
                 **,..          ....,,,,,,,....   ..,***,,,*#.         .,%@
                 ./,...       B l e n d e r B o t ...***,,,*#          .*%@
                  /*,..          ...,,,,,,,

07:59:47 | building dictionary first...
07:59:47 | Overriding opt["init_model"] to zoo:blender/blender_90M/model (previously: /usr/local/lib/python3.7/dist-packages/data/models/blender/blender_90M/model)
07:59:47 | Overriding opt["optimizer"] to adam (previously: mem_eff_adam)
07:59:47 | your model is being loaded with opts that do not exist in the model you are initializing the weights with: allow_missing_init_opts: False,loglevel: info,dynamic_batching: full,is_debug: False,final_extra_opt: ,eval_dynamic_batching: None,num_workers: 0,max_train_steps: -1,log_every_n_steps: 50,validation_every_n_steps: -1,world_logs: ,save_format: conversations,log_keep_fields: all,tensorboard_logdir: None,wandb_log: False,wandb_name: None,wandb_project: None,wandb_entity: None,mutators: None,fromfile_datapath: /content/drive/MyDrive/colabs/aliae-workspace/datasets/french_reddit/data,fromfile_datatype_extension: True,n_encoder_layers: -1,n_decoder_layers: -1,model_parallel: False,checkpoint_activations

In [ ]:
# mydrive_path = '/content/finetuned-multitask-400m-double-sided-2epochs'
# mydrive_path = '/content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/'

In [ ]:
# 400M settings
# !rm -rf $finetuned_model_path
!mkdir -p $finetuned_model_path


from parlai.scripts.train_model import TrainModel

TrainModel.main(
    # task
    task= "french_blended_skill_talk,french_xpersona,french_empathetic_dialogues",
    multitask_weights= "1,3,3",

    # task='fromfile:parlaiformat', 
    # fromfile_datapath='copied_dataset_french_bst/',
    # fromfile_datatype_extension=True,

    model='transformer/generator',
    model_file= f'{finetuned_model_path}/model',
    
    # initialize with a pretrained model
    init_model= init_model,
    dict_file=dict_file,
    
    # depend on your gpu
    validation_every_n_epochs=0.25, # veps= 0.25, 
    num_epochs = 5,
    log_every_n_secs= 60,
    verbose = True,
    attention_dropout= 0.0, 
    batchsize= 16, 
    fp16= True, fp16_impl= "mem_efficient",
    # save_after_valid= True,

    # arguments we get from the pretrained model. "from recipes page for 2.7B model" 
    embedding_size= 1280, ffn_size= 5120,
    variant= "prelayernorm",
    n_heads= 32, n_positions= 128, 
    n_encoder_layers= 2, n_decoder_layers= 12,

    label_truncate= 128, text_truncate= 128, truncate= 128,
    activation= "gelu",
    history_add_global_end_token= "end", 
    delimiter= '  ', 
    dict_tokenizer= "bytelevelbpe",
    dropout= 0.1,
    
    # some training arguments, specific to this fine-tuning
    lr= 7e-06, lr_scheduler= "reduceonplateau", lr_scheduler_patience= 3,
    optimizer= "mem_eff_adam",
    relu_dropout= 0.0, 
    model_parallel= False,
    warmup_updates= 100,
    update_freq= 2,
    gradient_clip= 0.1, 

    # speeds up validation
    skip_generation= True,
    vp= 10,
    validation_metric= "ppl", #vmt = "ppl"
    validation_metric_mode= "min", # vmm= "min"

    # customized parameters
    # inference = 'topk', 
    # temperature = 0.7, 
    # topk=30, 
    # beam_length_penalty=1.03
)

In [ ]:
# !cp -rv /content/finetuned-multitask-400m-double-sided-2epochs/* /content/drive/MyDrive/colabs/blender-models/finetuned-multitask-400m-double-sided/
# !ls -lah /content/finetuned-multitask-400m-double-sided-2epochs/
# !ls -lah /content/finetuned-multitask-400m-double-sided/

# 4.Display Model

In [ ]:
from parlai.scripts.display_model import DisplayModel
DisplayModel.main(
    task='french_blended_skill_talk',
    model_file= f'{finetuned_model_path}/model',
    num_examples=20,
    skip_generation=False,
)

15:06:36 | Overriding opt["task"] to french_blended_skill_talk (previously: french_blended_skill_talk,french_xpersona,french_empathetic_dialogues)
15:06:36 | Overriding opt["skip_generation"] to False (previously: True)
15:06:36 | Using CUDA
15:06:36 | loading dictionary from /content/drive/MyDrive/colabs/blender-models/finetuned-multitask-90m-double-sided/model.dict
15:06:36 | num words = 54944
15:06:38 | Total parameters: 87,508,992 (86,984,704 trainable)
15:06:38 | Loading existing model params from /content/drive/MyDrive/colabs/blender-models/finetuned-multitask-90m-double-sided/model
15:06:42 | creating task(s): french_blended_skill_talk
15:06:42 | Loading ParlAI text data: /content/dataset_french_bst/valid.txt
15:06:42 | Opt:
15:06:42 |     activation: gelu
15:06:42 |     adafactor_eps: '[1e-30, 0.001]'
15:06:42 |     adam_eps: 1e-08
15:06:42 |     add_p1_after_newln: False
15:06:42 |     aggregate_micro: False
15:06:42 |     allow_missing_init_opts: False
15:06:42 |     attentio

In [ ]:
# from parlai.scripts.display_data import DisplayData
# DisplayData.main(task='empathetic_dialogues', num_examples=10)

from parlai.scripts.display_data import DisplayData
DisplayData.main(
    task='fromfile:parlaiformat',
    fromfile_datapath='copied_dataset_french_bst/',
    fromfile_datatype_extension=True,
    # model_file= f'{finetuned_model_path}/model',
    # dict_file= dict_file,

    num_examples=20,
    # skip_generation=False,

    # beam_min_length= 20,
    # beam_block_ngram= 3,
    # beam_context_block_ngram= 3,
    # beam_size= 10,

    # inference= "beam"
)

12:46:14 | Opt:
12:46:14 |     allow_missing_init_opts: False
12:46:14 |     batchsize: 1
12:46:14 |     datapath: /usr/local/lib/python3.7/dist-packages/data
12:46:14 |     datatype: train:ordered
12:46:14 |     dict_class: None
12:46:14 |     display_add_fields: 
12:46:14 |     download_path: None
12:46:14 |     dynamic_batching: None
12:46:14 |     fromfile_datapath: copied_dataset_french_bst/
12:46:14 |     fromfile_datatype_extension: True
12:46:14 |     hide_labels: False
12:46:14 |     ignore_agent_reply: True
12:46:14 |     image_cropsize: 224
12:46:14 |     image_mode: raw
12:46:14 |     image_size: 256
12:46:14 |     init_model: None
12:46:14 |     init_opt: None
12:46:14 |     is_debug: False
12:46:14 |     loglevel: info
12:46:14 |     max_display_len: 1000
12:46:14 |     model: None
12:46:14 |     model_file: None
12:46:14 |     multitask_weights: [1]
12:46:14 |     mutators: None
12:46:14 |     num_examples: 20
12:46:14 |     override: "{'task': 'fromfile:parlaiformat', '